In [9]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def swish(x):
  return x*tf.nn.sigmoid(x)

def selu(x):
  with tf.name_scope('elu') as scope:
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale*tf.where(x>=0.0, x, alpha*tf.nn.elu(x))

def relu(x):
    return tf.nn.relu(x)

def activation(x):
#  return selu(x)
#  return relu(x)
#  return tf.nn.sigmoid(x)
#  return tf.nn.elu(x)
  return swish(x)

def initialize(shape, stddev=0.1):
  return tf.truncated_normal(shape, stddev=stddev)
  #return tf.zeros(shape)


# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

init_learning_rate = tf.placeholder(tf.float32)

# Create the model
L1_units_count = 100
x = tf.placeholder(tf.float32, [None, 784])

# exponetial lr decay
epoch_steps = tf.to_int64(tf.div(60000, tf.shape(x)[0]))
global_step = tf.train.get_or_create_global_step()
current_epoch = global_step//epoch_steps
decay_times = current_epoch #//2
current_learning_rate = tf.multiply(init_learning_rate, 
                                    tf.pow(0.575, tf.to_float(decay_times)))

W_1 = tf.Variable(initialize([784, L1_units_count], 
                             stddev=np.sqrt(2/784)))
b_1 = tf.Variable(tf.constant(0.001, shape=[L1_units_count]))
logits_1 = tf.matmul(x, W_1) + b_1
output_1 = activation(logits_1)

L2_units_count = 10 
W_2 = tf.Variable(initialize([L1_units_count, 
                              L2_units_count], 
                             stddev=np.sqrt(2/L1_units_count)))
b_2 = tf.Variable(tf.constant(0.001, shape=[L2_units_count]))
logits_2 = tf.matmul(output_1, W_2) + b_2  

y = logits_2



# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
l2_loss = tf.nn.l2_loss(W_1) + tf.nn.l2_loss(W_2)
total_loss = cross_entropy + 4e-5*l2_loss
train_step = tf.train.AdamOptimizer(
    current_learning_rate).minimize(
    total_loss, global_step=global_step)

# 还记得不？不要把graph定义写到图里面
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Train
for step in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  lr = 1e-2
  _, loss, l2_loss_value, total_loss_value, current_lr_value = \
      sess.run(
               [train_step, cross_entropy, l2_loss, total_loss, 
                current_learning_rate], 
               feed_dict={x: batch_xs, y_: batch_ys, 
                          init_learning_rate:lr})
  
  if (step+1) % 100 == 0:
    print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    #print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels}))
    #print(current_lr_value)
    

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
step 100, entropy loss: 0.223681, l2_loss: 290.696442, total loss: 0.235309
0.9329
step 200, entropy loss: 0.157771, l2_loss: 396.068665, total loss: 0.173613
0.9519
step 300, entropy loss: 0.172144, l2_loss: 487.638062, total loss: 0.191649
0.9552
step 400, entropy loss: 0.139544, l2_loss: 569.289001, total loss: 0.162315
0.9552
step 500, entropy loss: 0.107343, l2_loss: 653.666443, total loss: 0.133490
0.9622
step 600, entropy loss: 0.052174, l2_loss: 746.356445, total loss: 0.082028
0.9597
step 700, entropy loss: 0.021982, l2_loss: 735.011169, total loss: 0.051383
0.9702
step 800, entropy loss: 0.091634, l2_loss: 727.845947, total loss: 0.120748
0.9701
step 900, entropy loss: 0.074022, l2_loss: 724.336731,